In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [3]:
url = "https://raw.githubusercontent.com/maxxxxc/SIR-Summer-2023/main/dataset/default%20of%20credit%20card%20clients.csv"
df = pd.read_csv(url)

X = df.drop("Y", axis=1)
y = df["Y"]

y.value_counts()

0    23364
1     6636
Name: Y, dtype: int64

In [4]:
#divide

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
#param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}
#svm = SVC(kernel='rbf')

#grid_search = GridSearchCV(svm, param_grid, cv=5)
#grid_search.fit(X_train_scaled, y_train)

#best_params = grid_search.best_params_
#best_C = best_params['C']
#best_gamma = best_params['gamma']

best_C = 1
best_gamma = 1

#Fit the SVM with the best parameters:
svm = SVC(kernel='rbf', C=best_C, gamma=best_gamma)

In [8]:
svm.fit(X_train_scaled, y_train)

SVC(C=1, gamma=1)

In [9]:
# Make predictions on the test set
y_test_pred = svm.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", accuracy)

# Prediction rule is essentially: 1 if distance > 0
distances = svm.decision_function(X_test_scaled)
tmp = np.where(distances > 0, 1, 0)
print("Match between Prediction and Rule based on Distance")
accuracy_score(tmp, y_test_pred)

Test Accuracy: 0.79175
Match between Prediction and Rule based on Distance


1.0

In [10]:
from sklearn.calibration import CalibratedClassifierCV

In [11]:
svm_platt = CalibratedClassifierCV(svm)
svm_platt.fit(X_train_scaled, y_train)
y_test_prob = svm_platt.predict_proba(X_test_scaled)

In [12]:
y_test_pred = np.where(y_test_prob[:,1] > 0.5, 1, 0)
accuracy_score(y_test, y_test_pred)

0.7905

In [13]:
def svm_iterate_process(X, y):
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    
    # Define the number of batches
    num_batches = 11
    
    # Randomly shuffle the data indices
    indices = np.random.permutation(len(X_train))
    
    # Calculate the batch size
    batch_size = len(X_train) // num_batches
    
    # Make predictions on the test set using majority voting
    preds_voting = np.zeros(len(y_test))
    # Make predictions on the test set using average of logit
    preds_distance = np.zeros(len(y_test))
    #Make predictions on the test set using average of probs
    preds_prob = np.zeros(len(y_test))

    
    # Split the training data into batches, fit a logistic regression model on each batch
    for i in range(num_batches):
        # Calculate the starting and ending indices for the current batch
        start_index = i * batch_size
        end_index = (i + 1) * batch_size
        
        # Select the current batch for training
        X_batch = X_train.iloc[indices[start_index:end_index]]
        y_batch = y_train.iloc[indices[start_index:end_index]]
        
        scaler = StandardScaler()
        X_batch_scaled = scaler.fit_transform(X_batch)
        X_test_scaled = scaler.transform(X_test)
        
        
        # Create a support vector machine model
        param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}
        svm = SVC(kernel='rbf')

        grid_search = GridSearchCV(svm, param_grid, cv=5)
        grid_search.fit(X_batch_scaled, y_batch)

        best_params = grid_search.best_params_
        best_C = best_params['C']
        best_gamma = best_params['gamma']
        svm = SVC(kernel='rbf', C=best_C, gamma=best_gamma)
        
        # Fit the model on the current batch
        svm.fit(X_batch_scaled, y_batch)
        
               
        # Accumulate the predictions using majority voting
        y_pred = svm.predict(X_test_scaled)
        preds_voting += (y_pred == 1)

    
        # Accumulate the predictions using majority voting
        y_pred = svm.decision_function(X_test_scaled)
        preds_distance += y_pred
        
        
        #Accumulate the probs
        svm_platt = CalibratedClassifierCV(svm)
        svm_platt.fit(X_batch_scaled, y_batch)
        y_pred = svm_platt.predict_proba(X_test_scaled)
        preds_prob += y_pred[:,1]
   
    
    accuracy = np.zeros(4)
    
    # Majority voting (selecting the most frequent prediction for each sample)
    final_predictions = np.where(preds_voting > num_batches / 2, 1, 0)
    accuracy[0] = accuracy_score(y_test, final_predictions)
    # Average of logit
    final_predictions = np.where(preds_distance > 0, 1, 0)
    accuracy[1] = accuracy_score(y_test, final_predictions)
    #Average of probs
    final_predictions = np.where(preds_prob / num_batches > 0.5, 1, 0)
    accuracy[2] = accuracy_score(y_test, final_predictions)
    
    # Train a model on all 11 batches of training data
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    param_grid = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10]}
    svm = SVC(kernel='rbf')

    grid_search = GridSearchCV(svm, param_grid, cv=5)
    grid_search.fit(X_train_scaled, y_train)

    best_params = grid_search.best_params_
    best_C = best_params['C']
    best_gamma = best_params['gamma']
    svm = SVC(kernel='rbf', C=best_C, gamma=best_gamma)
    
    svm.fit(X_train_scaled, y_train)
    y_pred = svm.predict(X_test_scaled)
    accuracy[3] = accuracy_score(y_test, y_pred)
    
    return accuracy

In [14]:
# Number of times to repeat the process
num_repeats = 50

# Initialize an empty matrix (10-by-4) to store accuracies
accuracies = np.zeros((num_repeats, 4))

seed = 42
# Repeat the process and store accuracies
for i in range(num_repeats):
    np.random.seed(seed)
    accuracies[i] = svm_iterate_process(X, y)
    seed += 2
    

# Print the accuracies
print("Accuracies:", accuracies)

Accuracies: [[0.81191667 0.81116667 0.811      0.81575   ]
 [0.8175     0.81775    0.8145     0.81808333]
 [0.81566667 0.81608333 0.81475    0.81916667]
 [0.81616667 0.81525    0.81625    0.82025   ]
 [0.81841667 0.818      0.81533333 0.82258333]
 [0.81458333 0.81525    0.81183333 0.8195    ]
 [0.81766667 0.81566667 0.814      0.81791667]
 [0.81633333 0.81625    0.8145     0.81875   ]
 [0.81391667 0.81458333 0.81491667 0.81825   ]
 [0.81016667 0.81208333 0.81033333 0.81691667]
 [0.81675    0.81625    0.81425    0.82016667]
 [0.81641667 0.81575    0.81366667 0.81966667]
 [0.8085     0.80958333 0.80816667 0.81308333]
 [0.81591667 0.81583333 0.81491667 0.82183333]
 [0.81383333 0.81491667 0.81433333 0.8185    ]
 [0.81225    0.81266667 0.81183333 0.81641667]
 [0.81316667 0.81466667 0.81191667 0.81675   ]
 [0.81508333 0.81566667 0.81458333 0.81983333]
 [0.81341667 0.81508333 0.81275    0.81875   ]
 [0.81641667 0.81725    0.81408333 0.82108333]
 [0.81358333 0.814      0.81233333 0.81825   ]
 

In [19]:
np.mean(accuracies, axis = 0) 

array([0.81450667, 0.81473833, 0.81277833, 0.81845333])

In [20]:
np.std(accuracies, axis = 0)

array([0.00282355, 0.00258145, 0.00262512, 0.00256337])

In [15]:
y_test_prob[:5,:]

array([[0.8599092 , 0.1400908 ],
       [0.87752305, 0.12247695],
       [0.83075836, 0.16924164],
       [0.82243521, 0.17756479],
       [0.79854919, 0.20145081]])

In [16]:
from sklearn.linear_model import LogisticRegression
train_distances = svm.decision_function(X_train_scaled)
model = LogisticRegression(max_iter=500)
model.fit(train_distances.reshape(-1, 1), y_train)
tmp = model.predict_proba(distances.reshape(-1, 1))

In [17]:
tmp[:5,:]

array([[0.95123883, 0.04876117],
       [0.97694979, 0.02305021],
       [0.90887901, 0.09112099],
       [0.90105996, 0.09894004],
       [0.8436286 , 0.1563714 ]])

In [18]:
tmp1 = np.where(tmp[:,1] > 0.5, 1, 0)
accuracy_score(y_test, tmp1)

0.7880833333333334